<a href="https://colab.research.google.com/github/francrevillent/ng-startwars/blob/master/Pruebas_de_v3_2_VMO_y_LCL_Pedidos_Detalle_y_Agrupado_y_sin_iva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandasql

In [ ]:
"""
Created on Mon Mar 15 12:52:22 2021
Last_modified on Apr 14 2021

@author:  frodriguez
@reviewd: enasarrepuy
"""

"""
Crear un archivo en Google Colab que ejecute la instrucción necesaria para descargar un archivo diario con las ventas de los últimos 60 días (el número de días ahora mismo no es importante) y guarde ese documento en en una carpeta de drive.


El usuario de prestashop con solo permisos de ver (no escritura, para no joder nada)  es este


Empieza con este prestashop que es el negocio menos importante: 
https://www.ventamueblesonline.es/admin8623
e.nasarre.puy@gmail.com      Nasarre3030
"""

import requests
import json 
import pandas as pd 
import datetime

from google.colab import drive
import os 
drive.mount('/content/gdrive')



In [ ]:
class FranPrestashop():
    
    ## Datos globales para los métodos de la clase. Todo lo que se tenga que reutilizar entre métodos son atributos de la clase
    URL_HOME = None
    URL_LOGIN = None
    USUARIO = None
    PASS = None
 
    ## Aquí se almacenan los tokens después de la autenticación
    TOKENS = None
    COOKIES = None
    
    
    def __init__(self, URL_HOME, USUARIO, PASS):
        
        self.URL_HOME = URL_HOME 
        self.URL_LOGIN = self.URL_HOME + "index.php"
        self.USUARIO = USUARIO 
        self.PASS = PASS    
        
    def autenticacionPrestashop(self):
        """
        Se autentica con Prestashop, recupera los tokens necesarios y la cookie y la almacena como atributo de la clase
        """
        
        headers = {
                'accept': 'application/json, text/javascript, */*; q=0.01'
                , 'accept-encoding': 'gzip, deflate, br'
                , 'accept-language': 'es-ES,es;q=0.9'
                , 'cache-control': 'no-cache'
                , 'content-length': '192'
                , 'content-type': 'application/x-www-form-urlencoded; charset=UTF-8'
                }
        
        
        #### HACER LOGIN Y OBTENER EL TOKEN
        myobj = {
                "email": self.USUARIO
                , "passwd": self.PASS
                , "submitLogin": "1"
                , "controller": "AdminLogin"
                , "ajax": "1"
                }
        
        aux = requests.post(self.URL_LOGIN, data=myobj, headers=headers)
        respuesta = aux.text
        
        
        URL_AdminDashboard = self.URL_HOME + json.loads(respuesta).get("redirect")
        token_AdminDashboard = respuesta.split("&token=")[1].split("\"")[0]
        self.COOKIES = aux.cookies
        
        ### OBTENER LOS TOKENS que se encuentran dentro de la URL del Dashboard
        
        aux_2 = requests.get( URL_AdminDashboard, timeout=5, cookies=self.COOKIES)
        respuesta_2 = aux_2.text
        
        token = respuesta_2.split("var token = '")[1].split("'")[0]
        token_admin_orders = respuesta_2.split("var token_admin_orders = '")[1].split("'")[0]
        token_admin_customers = respuesta_2.split("var token_admin_customers = '")[1].split("'")[0]
        token_admin_customer_threads = respuesta_2.split("var token_admin_customer_threads = '")[1].split("'")[0]
        token_employee = respuesta_2.split("var employee_token = '")[1].split("'")[0]
    
        self.TOKENS = {
                        "token": token
                        , "token_admin_orders": token_admin_orders
                        , "token_admin_customers": token_admin_customers
                        , "token_admin_customer_threads": token_admin_customer_threads
                        , "token_employee": token_employee
            }        
        
        
    def getContenidoCSV(self, fecha_desde, fecha_hasta):
        """ Devuelve el String con todos los pedidos del CSV descargado. Sin parsear """
        
        token_admin_orders = self.TOKENS["token_admin_orders"]
        ## Hacer un set de los filtros antes de empezar a filtrar y recuperar la cookie
        params = {
                'controller':'AdminOrders'
                , 'exportorder':''
                , 'token': token_admin_orders
                , 'orderFilter_a!date_add[0]': fecha_desde
                , 'orderFilter_a!date_add[1]': fecha_hasta
                , 'order_pagination':'999999'
                , 'submitFilter':'1'
                , 'submitFilterorder':'1'
            }
        
        aux_3 = requests.post( self.URL_HOME, timeout=60, params=params, cookies=self.COOKIES )
        contenido_csv = aux_3.text
        
        return contenido_csv
        
    
    def getContenidoListaPedidosLuzConLed(self, fecha_desde, fecha_hasta):
 
        contenido_csv = self.getContenidoCSV(fecha_desde, fecha_hasta)
        array_lineas = contenido_csv.splitlines()
        lista_obj = list()
        for i in range(1, len(array_lineas)):
            
            obj_array = array_lineas[i].split(";")
            
            fecha = obj_array[9].replace("\"", "")
            fecha = datetime.datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
            fecha_dia = fecha.strftime("%Y-%m-%d")
            
            total = obj_array[6].replace("\"", "")
            total = total.replace(" €", "").replace(",", ".").replace(" ", "")
            total = float( total )
            obj = {
                "id": obj_array[0].replace("\"", "")
                ,"Referencia": obj_array[1].replace("\"", "")
                ,"Nuevo cliente": obj_array[2].replace("\"", "")
                ,"Entrega": obj_array[3].replace("\"", "")
                ,"Cliente": obj_array[4].replace("\"", "")
                ,"Empresa": obj_array[5].replace("\"", "")
                ,"Total": total /1.21
                ,"Pago": obj_array[7].replace("\"", "")
                ,"Estado": obj_array[8].replace("\"", "")
                ,"Fecha": fecha
                ,"fecha_dia": fecha_dia
                }
            lista_obj.append(obj)
 
        return lista_obj
    
    
    def getContenidoListaPedidosVentaMuebles(self, fecha_desde, fecha_hasta):

        contenido_csv = self.getContenidoCSV(fecha_desde, fecha_hasta)
        array_lineas = contenido_csv.splitlines()
        lista_obj = list()
        for i in range(1, len(array_lineas)):
            
            
            obj_array = array_lineas[i].split(";")
            fecha = obj_array[8].replace("\"", "")
            fecha = datetime.datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
            fecha_dia = fecha.strftime("%Y-%m-%d")
            
            total = obj_array[5].replace("\"", "")
            total = total.replace(" €", "").replace(",", ".").replace(" ", "")
            total = float( total )
            obj = {
                "id": obj_array[0].replace("\"", "")
                ,"Referencia": obj_array[1].replace("\"", "")
                ,"Phone": obj_array[2].replace("\"", "")
                ,"Entrega": obj_array[3].replace("\"", "")
                ,"Cliente": obj_array[4].replace("\"", "")
                ,"Total": total / 1.21
                ,"Pago": obj_array[6].replace("\"", "")
                ,"Estado": obj_array[7].replace("\"", "")
                ,"Fecha": fecha
                ,"fecha_dia": fecha_dia
                }
            lista_obj.append(obj)
 
        return lista_obj

## ***Funciones del código para descargar los dataframes del prestashop y para generar el dataframe agrupado con una consulta SQL***

In [ ]:
def getDataFramePrestashop( URL_HOME, USUARIO, PASS, CLIENTE = "LuzConLed" ):
    """
    Este código es común salvo el IF de la condición "CLIENTE". 
    Esta función crea la autenticación con prestashop, descarga el CSV filtrado, genera un dataframe y devuelve el dataframe
    """
        
    objetoFran = FranPrestashop( URL_HOME=URL_HOME, USUARIO=USUARIO, PASS=PASS )
    objetoFran.autenticacionPrestashop()
     
    ## Obtener el rango de fechas de los últimos 60 días
    fecha_hasta = datetime.datetime.utcnow()
    fecha_desde = fecha_hasta - datetime.timedelta(days=60) 
    fecha_hasta = fecha_hasta.strftime("%Y-%m-%d")
    fecha_desde = fecha_desde.strftime("%Y-%m-%d")
     
    
    if(CLIENTE == "LuzConLed"):
        print("Obteniendo los datos de LuzConLed")
        listaPedidos = objetoFran.getContenidoListaPedidosLuzConLed( fecha_desde, fecha_hasta )
    elif(CLIENTE == "ventaMuebles"):
        print("Obteniendo los datos de VentaMuebles")
        listaPedidos = objetoFran.getContenidoListaPedidosVentaMuebles( fecha_desde, fecha_hasta )
    

    df_pedidos = pd.DataFrame( listaPedidos )
    return df_pedidos
        
 


def generarDataframeAgrupadoPorWeb(df, tabla_sql):
    from pandasql import sqldf
    
    ## Obtener datos filtrados de los dataframes
    lista_estados = str( ( "Anticipo Pagado"
                               , "Entregado"
                               , "Entregado y cobrado por agencia"
                               , "Enviado"
                               , "ENVIADO DESDE FABRICA"
                               , "Enviado GLS"
                               , "Enviado Seur"
                               , "Enviado TXT"
                               , "Enviado TXT VLC"
                               , "Pago aceptado"
                               , "Pedido contra-reembolso"
                               , "Pedido en espera de producto" ) 
                             )
        
    lista_pagos = str( ( "Bizum"
                         , "Bonifico bancario"
                         , "Card payment REDSYS Virtual POS"
                         , "Pagamento contra entrega (C.O.D.)"
                         , "Pagamentos por cheque"
                         , "Pago Contrarrembolso"
                         , "PayPal avec frais"
                         , "PayPal con Recargo"
                         , "PayPal supplemento"
                         , "Sequra - Pago flexible"
                         , "SIBS MB WAY"
                         , "SIBS MULTIBANCO"
                         , "Transferencia bancaria"
                         , "Transferência bancária"
                         , "Virement bancaire" ) 
                       )
    
    consulta_sql = ("""
                       select 
                           fecha_dia
                           , sum( case 
                                     when (Entrega = 'España') then 
                                         Total
                                    end 
                              ) as España
                           , sum( case 
                                     when (Entrega = 'Francia') then 
                                         Total
                                    end 
                              ) as Francia
                           , sum( case 
                                     when (Entrega = 'Italia') then 
                                         Total
                                    end 
                              ) as Italia
                           , sum( case 
                                     when (Entrega = 'Portugal') then 
                                         Total
                                    end 
                              ) as Portugal
                           , sum ( case 
                                     when (Entrega = 'España') then 
                                         Total
                                     when (Entrega = 'Francia') then 
                                         Total
                                     when (Entrega = 'Italia') then 
                                         Total
                                     when (Entrega = 'Portugal') then 
                                         Total
                                     end
                               ) as Total
                       from {DATAFRAME}
                       where 1 = 1 
                           and Estado in """+ lista_estados +""" 
                           and Pago in """+ lista_pagos +""" 
                       group by fecha_dia
                       order by fecha_dia
                    """).replace("{DATAFRAME}", tabla_sql)
                    
    output = sqldf(consulta_sql)
    return output  

def generarDataframeAgrupadoPorMarket(df, tabla_sql):
    from pandasql import sqldf
    
    ## Obtener datos filtrados de los dataframes
    lista_estados = str( ( "Anticipo Pagado"
                               , "Entregado"
                               , "Entregado y cobrado por agencia"
                               , "Enviado"
                               , "ENVIADO DESDE FABRICA"
                               , "Enviado GLS"
                               , "Enviado Seur"
                               , "Enviado TXT"
                               , "Enviado TXT VLC"
                               , "Pago aceptado"
                               , "Pedido contra-reembolso"
                               , "Pedido en espera de producto" ) 
                             )
        
    lista_pagos = str( ( "Bizum"
                         , "Bonifico bancario"
                         , "Card payment REDSYS Virtual POS"
                         , "Pagamento contra entrega (C.O.D.)"
                         , "Pagamentos por cheque"
                         , "Pago Contrarrembolso"
                         , "PayPal avec frais"
                         , "PayPal con Recargo"
                         , "PayPal supplemento"
                         , "Sequra - Pago flexible"
                         , "SIBS MB WAY"
                         , "SIBS MULTIBANCO"
                         , "Transferencia bancaria"
                         , "Transferência bancária"
                         , "Virement bancaire" ) 
                       )
    
    consulta_sql = ("""
                       select 
                           fecha_dia
                           , sum( case 
                                     when (Entrega = 'España') then 
                                         Total
                                    end 
                              ) as España
                           , sum( case 
                                     when (Entrega = 'Francia') then 
                                         Total
                                    end 
                              ) as Francia
                           , sum( case 
                                     when (Entrega = 'Italia') then 
                                         Total
                                    end 
                              ) as Italia
                           , sum( case 
                                     when (Entrega = 'Portugal') then 
                                         Total
                                    end 
                              ) as Portugal
                           , sum ( case 
                                     when (Entrega = 'España') then 
                                         Total
                                     when (Entrega = 'Francia') then 
                                         Total
                                     when (Entrega = 'Italia') then 
                                         Total
                                     when (Entrega = 'Portugal') then 
                                         Total
                                     end
                               ) as Total
                       from {DATAFRAME}
                       where 1 = 1 
                           and Estado in """+ lista_estados +""" 
                           and Pago in """+ lista_pagos +""" 
                       group by fecha_dia
                       order by fecha_dia
                    """).replace("{DATAFRAME}", tabla_sql)
                    
    output = sqldf(consulta_sql)
    return output  



## **LUZCONLED**
Comprobar si existe o no la carpeta, en el caso de que no exista la carpeta la crea.

Después descarga el raw data con las ventas de los últimos 60 días, de ese raw data descargado obtiene las ventas agrupadas con las condiciones aplicadas.

Los dos datasets son exportados a la carpeta que se encuentra en la variable PATH_CSV

In [ ]:
PATH_CSV = "/content/gdrive/My Drive/LCL/LCL - 2021/LCL - 2021 - 04/"
#PATH_CSV = "/content/gdrive/My Drive/PedidosEnriqueNasarre/"

if not os.path.exists(PATH_CSV):
    os.makedirs(PATH_CSV)

In [ ]:
## Pedidos de https://www.luzconled.es
######################################
######################################
fecha_hasta = datetime.datetime.utcnow().strftime("%Y-%m-%d")


URL_HOME = "https://www.luzconled.es/admin3257/"
USUARIO = "e.nasarre.puy@gmail.com"
PASS = "Nasarre3030"
 

df_luz = getDataFramePrestashop( URL_HOME, USUARIO, PASS, CLIENTE = "LuzConLed" )
df_luz_agg = generarDataframeAgrupadoPorWeb(df = df_luz, tabla_sql = "df_luz")
df_luz_market = generarDataframeAgrupadoPorMarket(df = df_luz, tabla_sql = "df_luz")


PATH_SALIDA = ("pedidos_luzconled_{fecha_hasta}.xls").replace("{fecha_hasta}", fecha_hasta)
df_luz.to_excel(PATH_CSV + PATH_SALIDA)

PATH_SALIDA_LAST = ("pedidos_luzconled_{fecha_hasta}.xls").replace("{fecha_hasta}", "last")
df_luz.to_excel(PATH_CSV + PATH_SALIDA_LAST)

PATH_SALIDA_AGG = ("pedidos_luzconled_agg_{fecha_hasta}.xls").replace("{fecha_hasta}", fecha_hasta)
df_luz_agg.to_excel(PATH_CSV + PATH_SALIDA_AGG)


PATH_SALIDA_MARKET = ("pedidos_luzconled_market_{fecha_hasta}.xls").replace("{fecha_hasta}", fecha_hasta)
df_luz_market.to_excel(PATH_CSV + PATH_SALIDA_MARKET)


Obteniendo los datos de LuzConLed


## **VENTAMUEBLESONLINE**
Comprobar si existe o no la carpeta, en el caso de que no exista la carpeta la crea.

Después descarga el raw data con las ventas de los últimos 60 días, de ese raw data descargado obtiene las ventas agrupadas con las condiciones aplicadas.

Los dos datasets son exportados a la carpeta que se encuentra en la variable PATH_CSV

In [ ]:
PATH_CSV = "/content/gdrive/My Drive/VMO/VMO - 2021/VMO - 2021 - 04/"
#PATH_CSV = "/content/gdrive/My Drive/PedidosEnriqueNasarre/"

if not os.path.exists(PATH_CSV):
    os.makedirs(PATH_CSV)

In [ ]:
## Pedidos de https://www.ventamueblesonline.es
######################################
######################################
fecha_hasta = datetime.datetime.utcnow().strftime("%Y-%m-%d")


URL_HOME = "https://www.ventamueblesonline.es/admin8623/"
USUARIO = "e.nasarre.puy@gmail.com"
PASS = "Nasarre3030"


df_muebles = getDataFramePrestashop( URL_HOME, USUARIO, PASS, CLIENTE = "ventaMuebles" )
df_muebles_agg = generarDataframeAgrupadoPorWeb(df = df_muebles, tabla_sql = "df_muebles")
df_muebles_market = generarDataframeAgrupadoPorMarket(df = df_muebles, tabla_sql = "df_muebles")


PATH_SALIDA = ("pedidos_ventamueblesonline_{fecha_hasta}.xls").replace("{fecha_hasta}", fecha_hasta)
df_muebles.to_excel(PATH_CSV + PATH_SALIDA)

PATH_SALIDA_AGG = ("pedidos_ventamueblesonline_agg_{fecha_hasta}.xls").replace("{fecha_hasta}", fecha_hasta)
df_muebles_agg.to_excel(PATH_CSV + PATH_SALIDA_AGG)

PATH_SALIDA_MARKET = ("pedidos_ventamueblesonline_market_{fecha_hasta}.xls").replace("{fecha_hasta}", fecha_hasta)
df_muebles_market.to_excel(PATH_CSV + PATH_SALIDA_MARKET)





Obteniendo los datos de VentaMuebles
